# Setup

In [ ]:
# Install all necessary packages
using Distributed
@everywhere using Pkg; @everywhere Pkg.activate(".")
Pkg.instantiate()
using ActiveSwingAsymmetricWalkingIncreaseTrunkKinematicVariability, Biomechanics, ProgressMeter

In [ ]:
# Add workers for parallel processing
prs = addprocs(2;topology=:master_worker, exeflags=["-O3", "--project=@."])

# Load relevant code on all processes
@everywhere using ActiveSwingAsymmetricWalkingIncreaseTrunkKinematicVariability

In [ ]:
trials = readtrials("data");

In [ ]:
[ (Subject=r[1], numtrials=length(r[2])) for r in [ (sub, findall(t -> t.subject == sub, trials)) for sub in 1:15 ] ]

## Main analysis

In [ ]:
# Setup progressmeter and lock
pdesc = "Processing data... "
p = Progress(length(trials)+1; desc=pdesc, barglyphs=BarGlyphs("[=>.]"))
uplock = ReentrantLock()

In [ ]:
# Update the progressmeter in a thread-safe manner
@everywhere function updateprogress()
    lock(uplock)
    next!(p)
    unlock(uplock)
    nothing
end

# This is used by the workers
@everywhere function analyzeandupdate(trial)
    numstrides = 100
    atrial = analyzetrial(trial, numstrides)
    
    # Tell the master process to update the progressmeter
    remotecall_wait(updateprogress,1)
    return atrial
end

In [ ]:
# Fit all the perturbations, don't quit on an error, add it to the results
next!(p)
analyzedtrials = pmap(analyzeandupdate, trials; on_error=identity)
finish!(p)

In [ ]:
using Biomechanics

In [ ]:
# Check to see if any perturbations failed (to go back and see why they failed)
badtrials = findall(x -> !isa(x, AnalyzedSegment), analyzedtrials)
if !isempty(badtrials)
    @show badtrials
    @show analyzedtrials[badtrials]
#     variables = [
#         :left_steplength,
#         :SD_left_steplength,
#         :right_steplength,
#         :SD_right_steplength,
#         :left_steptime,
#         :right_steptime,
#         :SD_left_steptime,
#         :SD_right_steptime,
#         :stepwidth,
#         :SD_stepwidth,
#         :lvmean,
#         :avmean,
#         :lvstd,
#         :avstd,
#         :lvmax,
#         :avmax,
#         :WBAM_mean,
#         :WBAM_std
#     ]
#     # NAN the bad trials
#     analyzedtrials[badtrials] = [ AnalyzedSegment(Segment(trial, Dict{Symbol,Vector}(), SteadyStateSeg()), 
#                                                           Dict{Symbol,Any}(( (var, NaN) for var in variables ))) 
#                                   for trial in trials[badtrials] ]
end

In [ ]:
# We don't need the other workers anymore
rmprocs(prs)

## Results printing setup

In [ ]:
using Statistics, Dates, DelimitedFiles

In [ ]:
# Setup loop variables
scalarvars = [
    :left_steplength,
    :SD_left_steplength,
    :right_steplength,
    :SD_right_steplength,
    :left_steptime,
    :right_steptime,
    :SD_left_steptime,
    :SD_right_steptime,
    :stepwidth,
    :SD_stepwidth
]

vectorvars = [
    :lvmean,
    :avmean,
    :lvstd,
    :avstd,
    :lvmax,
    :avmax,
    :WBAM_mean,
    :WBAM_std
]

armconds = [ :held, :norm, :active ]
shortarms = Dict(
    :held => "_he",
    :norm => "_nr",
    :active => "_ac"
)

symconds = [ :sym, :asym ]
shortsym = Dict(
    :sym => "_sy,",
    :asym => "_as,"
)

subs = 1:15
numsubs = length(subs)

# Initialize the results string
results = Vector{String}(undef, 1)
results[1] = "Analysis of trunk kinematics under different arm swing and gait symmetry conditions\n"

In [ ]:
# Loop through all variables and conditions
io = IOBuffer()
for vari in eachindex(scalarvars)
    subresults = fill(",", 4)
    R = collect(1:15)
    _r = Array{Float64}(undef, numsubs)

    for arms in eachindex(armconds), symmetry in eachindex(symconds)
        # Only print the variable/condition if it is the first of its type
        subresults[1] *= prod([ arms, symmetry ] .== ones(Int,2)) ? string(scalarvars[vari])*"," : ","
        subresults[2] *= (symmetry == one(Int)) ? string(armconds[arms], ",") : ","
        subresults[3] *= string(symconds[symmetry], ",")
        subresults[4] *= string(scalarvars[vari])*shortarms[armconds[arms]]*shortsym[symconds[symmetry]]

        # Find all trials for this combination of conditions
        relevant = findall(analyzedtrials) do at
            at.s.trial.conds[:arms] == armconds[arms] &&
            at.s.trial.conds[:sym] == symconds[symmetry]
        end
        
        for sub in subs
            # Find the trial for this particular subject, convert it back to the absolute indices of `analyzedtrials`
            # `findfirst` to return a scalar (we know that there should/will only be 1 result)
            t = relevant[findfirst(at -> at.s.trial.subject == sub, analyzedtrials[relevant])]

            _r[sub] = analyzedtrials[t].results[scalarvars[vari]]
        end
        
        R = [ R _r ]
    end

    writedlm(io, R, ',')

    results = [results; "\n\n"; subresults; String(take!(io))]
end

In [ ]:
# Loop through all variables and conditions
io = IOBuffer()
for vari in eachindex(vectorvars)
    subresults = fill(",", 4)
    R = collect(1:numsubs)
    _r = Array{Float64}(undef, numsubs, 3)

    for arms in eachindex(armconds), symmetry in eachindex(symconds)
        # Only print the variable/condition if it is the first of its type
        subresults[1] *= prod([ arms, symmetry ] .== ones(Int,2)) ? string(vectorvars[vari])*"," : ","
        subresults[2] *= (symmetry == one(Int)) ? string(armconds[arms], ",,,") : ",,,"
        subresults[3] *= string(symconds[symmetry], ",,,")
        subresults[4] *= "X,Y,Z,"

        # Grab all the perturbations for this combination of conditions
        relevant = findall(analyzedtrials) do at
            at.s.trial.conds[:arms] == armconds[arms] &&
            at.s.trial.conds[:sym] == symconds[symmetry]
        end

        for sub in subs
            # Find the trial for this particular subject, convert it back to the absolute indices of `analyzedtrials`
            # `findfirst` to return a scalar (we know that there should/will only be 1 result)
            t = relevant[findfirst(at -> at.s.trial.subject == sub, analyzedtrials[relevant])]

            _r[sub,:] .= vec(analyzedtrials[t].results[vectorvars[vari]])
        end

        R = [R _r]
    end
    
    writedlm(io, R, ',')
    
    results = [results; "\n\n"; subresults; String(take!(io)) ]
end

## Write to file

In [ ]:
(path, io) = mktemp()

for line in results
    println(io, line)
end

close(io)

resfn = abspath("results.csv")

mv(path, resfn; force=true)